In [ ]:
import matplotlib.pyplot as plt
import pytesseract
import requests

import sys
sys.path.append("../01_crop_image")
from get_clean_image import *

import sys
sys.path.append("../02_extract_digits")
from get_digits import *

In [ ]:
def file2files(fpath, d):
    img = cv2.imread(fpath.as_posix(), cv2.IMREAD_GRAYSCALE)
    img = rotate_image(img)
    img = fix_persepective(img)
    img = crop_image(img)
    img = cv2.medianBlur(img,9)

    rois = to_digit_images(img)
    for i, digit_img in enumerate(rois):
        rd = ~digit_img
        n = pytesseract.image_to_string(rd, config='--oem 3 --psm 10 -c tessedit_char_whitelist=0123456789').replace('\x0c','').replace('\n','')
        if n == "": n = 'X'
        outfilepath = '%s/%s_%d_%s.png'%(d,n, i, fpath.stem)
        #print(outfilepath)
                           
        cv2.imwrite(outfilepath, digit_img)

In [ ]:
def batch(meter_dir='./meter_image', digit_dir='./digits_image/'):
    p = pathlib.Path(meter_dir)
    d = pathlib.Path(digit_dir)
    paths = p.glob('*.jpg')
    i=0
    for fpath in paths:
        print('  ...processing', fpath.name)
        try:
            file2files(fpath, d)
        except OCRError:
            print('     [OCR ERROR]', fpath)
            continue
        #if i>1: break
        i += 1
    print("processed %d files."%i )

if __name__ == '__main__':
    batch()
